In [12]:
## 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime 
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")


In [13]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [16]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断抽取“公告日期”的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 5419.49it/s]


0.4583333333333333

100%|██████████| 8660/8660 [00:01<00:00, 7358.61it/s]


In [17]:
train_df['sample_id']


7412     9090
7783     9601
7639     9362
5524     6761
6109     7442
        ...  
2161     2568
6876     8387
2430     2958
8828    10971
8231    10169
Name: sample_id, Length: 7217, dtype: int64

In [18]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
CN_site={'公司'}
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        for key in CN_site:
            if key in i:
                return i

       
val_gm =pd.DataFrame()
val_gm['sample_id'] = val_df['sample_id']
val_gm['predict_gm']=val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
test_gm = train_outputs.groupby('sample_id').apply(lambda row:list(row['实际购买公司名称'])[0]).reset_index()
test_gm.columns=['sample_id','gm']

# 判断抽取“实际购买公司名称”字段的准确率
val_gm = pd.merge(val_gm, test_gm, on='sample_id', how='left')
np.sum(val_gm['predict_gm'].astype(str) == val_gm['gm'].astype(str))/len(val_gm)

100%|██████████| 1800/1800 [00:00<00:00, 5292.28it/s]


0.7016666666666667

In [25]:
#text=text.replace(r'[ ]+',' ').replace('\r','^')
def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
 
    for item in title_num_char:
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[\^]")
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[\^]")
        #pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        #pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()[]（）【】][]")#*?[\^]
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    # for item in title_list:
    for item in s_title_num_char:
        # pattern = re.compile(item+r"[ ]*?[ ]*?[\d][^ ]+?:?[ ]?") 
        # pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5\^ ]*")
        # pattern = re.compile(item+'[0-9\u4e00-\u9fa5()[]（）【】*?[\^]]')*?[\^]
    #    pattern = re.compile(item+r"[\d]*?\u4e00-\u9fa5+[ ]")
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5\"(\.*?)\" ()\[\]（）【】：:]*?[\^]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

#from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","^").replace(r'[ ]+',' ')
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)`
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        flag=True
        for item1 in title_pos_words:
            if re.search(item1,title_des) is not None:
                flag=False
                break
        if flag:
            for item in title_neg_words:
                if re.search(item,title_des) is not None:
                    neg_index.append(index)
                    break
        index+=1
    return title_1_df.reset_index(drop=True).drop(neg_index)
    # print(title_list)

def get_judge_title_result(val_df):

    judge_title_result=None


    for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
        # print(sample_id)
        # print(text)
        judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])
    
    return judge_title_result

title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]#"含本次","含本公告","公司及控股子公司",]#,"继续使用","新购买","近期","信托","计划","项目",'']
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","意见","十二个月内","公告前","报备文件","前期","截至","到期","已到期",'前次',"控制","本公告日前","分析","披露","主体","关联","条款","合同","说明","协议"]
val_judge_title_result=get_judge_title_result(val_df)
#test_judge_title_result=get_judge_title_result(test_df)
# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)

100%|██████████| 1800/1800 [00:14<00:00, 122.65it/s]


In [ ]:
###9月11日
#val_judge_title_result.head(100)
def get_content_df(val_judge_title_result):
    sample_id=val_judge_title_result['sample_id'].drop_duplicates() 
    text_list=[]
    text_sampleid_list=[]
    content_df=pd.DataFrame()
    for i in tqdm(sample_id):
        text_df=val_judge_title_result[val_judge_title_result['sample_id']==i]
    #  text_df.iloc[0,5]
        #text_df
        n=len(text_df)
        text_sampleid_list.append(i)
        text=''
        for y in range(0,n):
            text=text+text_df.iloc[y,1]+text_df.iloc[y,5]
            text=re.sub("[ ]+"," ",text).replace("（","(").replace("）",")")
        #text
        text_list.append(text)
    content_df["sample_id"]=text_sampleid_list
    content_df["text"]=text_list
    return content_df.reset_index(drop=True)


val_content_df=get_content_df(val_judge_title_result)
train_content_df=get_content_df(train_judge_title_result)
test_content_df=get_content_df(test_judge_title_result)

val_content_df=pd.merge(val_content_df,val_df[["sample_id","tabel"]],on="sample_id")
# train_content_df=pd.merge(train_content_df,train_df[["sample_id","tabel"]],on="sample_id")
# test_content_df=pd.merge(test_content_df,test_df[["sample_id","tabel"]],on="sample_id")

'a a'

In [26]:
import re

def product_loc(df):
    #loc_df=pd.DataFrame()
   # loc_list=[]
   # df1=train_df[train_df['sample_id'].isin([df['sample_id']])]
  #  if len(df1['text'].values)==0:
  #      return None
  #  text=df1['text'].values[0]
   # text=text.replace('\n','').replace('  ',' ').replace('    ',' ').replace('\r','^')
   # lccp=df['理财产品名称']
  #  regex=re.compile(lccp)
  #  init_start=0
 #   init_end=0
    loc_df['理财产品名称']=df['理财产品名称']
    loc_df['认购金额(万元)']=df['认购金额(万元)']
    loc_df['sample_id']=df['sample_id']
    #loc_num=regex.findall(text)
 #   if loc_num is None:
   #      return None
 #   for x in range(0,len(loc_num)):
 #       loc=regex.search(text)
 #       if loc is None:
 #          continue
 #       loc_list.append('['+str(loc.start()+init_end)+"  "+str(loc.end()+init_end)+']')
 #       init_start=loc.start()
  #      init_end=loc.end()
 #       text=text[loc.end():]
  #  loc_df['位置']=str(loc_list)
    return loc_df
info_df=train_outputs.iloc[10]
loc_info=product_loc(info_df)
loc_info


,理财产品名称,认购金额(万元),sample_id


In [34]:
    df=train_outputs.iloc[10]
    df
    print('-----')
    loc_df['理财产品名称']=df['理财产品名称']
    loc_df['认购金额(万元)']=df['认购金额(万元)']
    loc_df['sample_id']=df['sample_id']
    loc_df
    df

sample_id                          1
认购日期             2019-03-15 00:00:00
理财产品名称                 招商银行日益月鑫90180
产品发行方名称                         招商银行
理财类型                          银行理财产品
认购金额(万元)                         100
产品起息日            2019-03-15 00:00:00
产品到息日            2019-09-11 00:00:00
产品期限                            180天
资金来源                            自有资金
实际购买公司名称                  恒生电子股份有限公司
实际购买公司和上市公司关系                   公司本身
买卖方是否有关联关系                         否
公告日期             2019-04-25 00:00:00
Name: 10, dtype: object

,理财产品名称,认购金额(万元),sample_id


sample_id                          1
认购日期             2019-03-15 00:00:00
理财产品名称                 招商银行日益月鑫90180
产品发行方名称                         招商银行
理财类型                          银行理财产品
认购金额(万元)                         100
产品起息日            2019-03-15 00:00:00
产品到息日            2019-09-11 00:00:00
产品期限                            180天
资金来源                            自有资金
实际购买公司名称                  恒生电子股份有限公司
实际购买公司和上市公司关系                   公司本身
买卖方是否有关联关系                         否
公告日期             2019-04-25 00:00:00
Name: 10, dtype: object

In [146]:
loc_df.iloc[1]

理财产品名称                丰和日利
认购金额(万元)               350
位置           (4649, 11345)
sample_id                1
Name: 3, dtype: object

In [138]:
df1=train_df[train_df['sample_id'].isin([14])]
#df1['text'].values[0]
df1['text'].values

array(["['证券代码：603337  证券简称：杰克股份  公告编号：2017-010 \\n杰克缝纫机股份有限公司 \\n关于公司使用自有闲置资金购买理财产品的实施公告 \\n \\n本公司董事会及全体董事保证本公告内容不存在任何虚假记载、误导\\n性陈述或者重大遗漏，并对其内容的真实性、准确性和完整性承担个别及\\n连带责任。 \\n杰克缝纫机股份有限公司（以下简称“公司”）于 2017 年 1 月 7 日召开\\n2017 年第一次临时股东大会，审议通过了《关于公司闲置自有资金购买理财\\n的议案》，为提高公司资金使用效率，合理利用自有资金，最大限度地发挥短\\n期富余自有资金的作用，在不影响正常经营及风险可控的前提下，公司拟使用\\n自有闲置资金购买低风险理财产品，公司使用累计不超过人民币 70,000 万元\\n的闲置资金购买理财产品。在该额度内，资金可以滚动使用。购买理财产品以\\n公司名义进行，公司董事会授权公司财务总监在上述额度范围内行使具体决策\\n权，包括但不限于：选择合格专业理财机构作为受托方、明确理财金额、期间、\\n选择理财产品品种、签署合同及协议等。本项授权有效期自 2017 年第一次临\\n时股东大会通过之日起至 2017 年年度股东大会召开之日止。 \\n根据日常经营资金需求，2017 年 2 月 23 日，公司与中国银行股份有限公\\n司台州市分行签署协议，使用 20,000.00 万元闲置自有资金购买中国银行股份\\n有限公司发行的“中银智富理财计划 2017 年第【014-1-HQ】期”非保本浮动型\\n收益人民币理财产品，并已办理完成相关手续。现将具体情况公告如下： \\n一、本次购买理财产品的基本情况 ', '（1）产品名称：中银智富理财计划2017年第【014-1-HQ】期； \\n（2）产品代码：AMZYZF2017014-1-HQ； \\n（3）产品类型：非保本浮动收益型； \\n（4）投资收益起算日：2017年2月23日； \\n（5）理财计划到期日：2017年8月23日； \\n（6）理财计划存续期限：181天（若本产品提前终止，以本产品实际存续\\n天数为准） \\n（7）产品费率：销售服务费率0.05%（年率）；理财资产托管及保管费0.05%\\n（年率）；超过预期年化投资收益率的收益作为理财计划管理人

In [157]:
np.set_printoptions(threshold=np.inf)
pd.set_option('max_colwidth',100)


In [ ]:
# 将table转换格式以及处理
def deal_tabel(row):
    row = eval(row)
    if len(row)==0:
        return []
    else:
        new_row = []
        for i in row:
            for d in i:
                new_temp = []
                for h in d:
                    # 这里处理空数据或者错误的数据
                    h = str(h).replace('None', '').replace('\n','').replace(' ', '')                    
                    if h=='':
                        continue
                    if h=='.':
                        continue
                    if h=='/':
                        continue
                    new_temp.append(h)
                new_row.append(new_temp)
        # 这里判断是否构成一个完整得认购数据(通过一个list进行判断)
        new_new_row = []
        for i in new_row:
            if len(i) == 0:
                continue
            elif len(i) <= 4:
                continue
            else:
                new_new_row.append(i)
        return new_new_row
train_df_tabel = train_df['tabel'].progress_apply(lambda row:deal_tabel(row))
val_df_tabel = val_df['tabel'].progress_apply(lambda row:deal_tabel(row))
test_df_tabel = test_df['tabel'].progress_apply(lambda row:deal_tabel(row))

In [21]:
#### 3.清洗提取出来的tabel数据，主要是清洗掉有问题的列 

#### 4.抽取的是单独的数据包含
#### 起息日，到息日， 金额，认购日期，产品发行方，理财产品

In [142]:
# 直接提取时间
# 如果出现两个时间第一个就是起息日，第二个就是到期日
# 如果出现一个时间就是起息日
# 出现的第一个money就是最后的金额
# 从这里面抽取所有序列
# 这里认为有逗号出现的就是money

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

from src.time_extractor import TimeFinder
import datetime
def get_list_data(df):
    df = list(df)
    new_df = []
    for i in tqdm(df):
        temp_df = []
        for h in i:
            new_h = []
            for digital in h:
                if ',' in digital:
                    # 这里也是为了统一数据有些是用元，有些是用万元
                    try:
                        ttt = float(digital.replace(',', '').replace('万元', '').replace('人民币', '').replace('元', ''))
                    except Exception:
                        continue
                    if ttt > 20000:
                        ttt = ttt/10000
                    new_h.append(ttt)
                else:
                    continue
            if len(new_h) == 0:
                continue
            temp_single = {}
            a = '_'.join(h)
            # 抽取时间和money
            t = TimeFinder()
            time_all = t.find_time(a)
            if time_all == None:
                continue
            rgrq = time_all[0]
            cpqxr = time_all[0]
            if len(time_all) > 1:
                try:
                    cpdxr = time_all[1]
                    # 相减
                    d1 = datetime.datetime.strptime(cpqxr, '%Y-%m-%d')
                    d2 = datetime.datetime.strptime(cpdxr, '%Y-%m-%d')
                    d = d2 - d1
                    cpqx = str(d.days) + '天'
                except Exception:
                    cpdxr = np.nan
                    cpqx = np.nan
            else:
                cpdxr = np.nan
                cpqx = np.nan
                
            # 筛选出除开数字与包含时间的列
            # 末尾是
            last_two = ['公司', '银行', '信托', '证券',  '分行', '支行', '中心', '业部', '商行', '建行']
            mowei = np.nan
            selected_bank_and_works = []
            for l in h:
                new_l = list(str(l))
                new_l_test = ''.join(l[-2:])
                if new_l_test in last_two:
                    mowei = l
                    continue
                if '资金' in l:
                    continue
                if '收益' in l:
                    continue
                if '到期' in l:
                    continue
                if ',' in l:
                    continue
                if '.' in l:
                    continue
                if '/' in l:
                    continue
                if '年' in l:
                    continue
                if '-' in l:
                    continue
                if len(l) < 4:
                    continue
                if is_number(l):
                    continue
                selected_bank_and_works.append(l)
            if len(selected_bank_and_works) < 1:
                continue
            
            temp_single['认购日期'] = rgrq
            temp_single['产品起息日'] = cpqxr
            temp_single['产品到期日'] = cpdxr
            temp_single['产品期限'] = cpqx
            temp_single['认购金额(万元)'] = new_h[0]
            temp_single['产品发行方名称'] = mowei
            temp_single['理财产品名称'] = selected_bank_and_works[0]
            temp_df.append(temp_single)
        new_df.append(temp_df)
    return new_df

val_contain_date = get_list_data(val_df_tabel)
test_contain_data = get_list_data(test_df_tabel) 

 55%|█████▍    | 4754/8660 [10:22<08:31,  7.63it/s]


KeyboardInterrupt: 

#### 5.汇总整理数据

In [138]:
# 将前面提取到的数据整理成对应格式
sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(val_df['sample_id'])
gg = list(val_gm)
time = list(val_time)
for i, value in enumerate(sample_id):
    for j in val_contain_date[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
val_result = result

NameError: name 'val_contain_date' is not defined

In [137]:
sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(test_df['sample_id'])
gg = list(test_gm)
time = list(test_time)
for i, value in enumerate(sample_id):
    for j in test_contain_data[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
test_result = result
test_result

NameError: name 'test_contain_data' is not defined